<a href="https://colab.research.google.com/github/nstaudac/DS-Unit-1-Sprint-3-Statistical-Tests-and-Experiments/blob/master/Nicholas_Staudacher_LS_DS_131_Statistics_Probability_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 1 Sprint 3 Assignment 1*

# Apply the t-test to real data

Your assignment is to determine which issues have "statistically significant" differences between political parties in this [1980s congressional voting data](https://archive.ics.uci.edu/ml/datasets/Congressional+Voting+Records). The data consists of 435 instances (one for each congressperson), a class (democrat or republican), and 16 binary attributes (yes or no for voting for or against certain issues). Be aware - there are missing values!

Your goals:

1. Load and clean the data (or determine the best method to drop observations when running tests)
2. Using hypothesis testing, find an issue that democrats support more than republicans with p < 0.01
3. Using hypothesis testing, find an issue that republicans support more than democrats with p < 0.01
4. Using hypothesis testing, find an issue where the difference between republicans and democrats has p > 0.1 (i.e. there may not be much of a difference)

Note that this data will involve *2 sample* t-tests, because you're comparing averages across two groups (republicans and democrats) rather than a single group against a null hypothesis.

Stretch goals:

1. Refactor your code into functions so it's easy to rerun with arbitrary variables
2. Apply hypothesis testing to your personal project data (for the purposes of this notebook you can type a summary of the hypothesis you formed and tested)

In [1]:
### YOUR CODE STARTS HERE

# importing raw data
import pandas as pd
voting_data_url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/voting-records/house-votes-84.data'
!curl https://archive.ics.uci.edu/ml/machine-learning-databases/voting-records/house-votes-84.data

republican,n,y,n,y,y,y,n,n,n,y,?,y,y,y,n,y
republican,n,y,n,y,y,y,n,n,n,n,n,y,y,y,n,?
democrat,?,y,y,?,y,y,n,n,n,n,y,n,y,y,n,n
democrat,n,y,y,n,?,y,n,n,n,n,y,n,y,n,n,y
democrat,y,y,y,n,y,y,n,n,n,n,y,?,y,y,y,y
democrat,n,y,y,n,y,y,n,n,n,n,n,n,y,y,y,y
democrat,n,y,n,y,y,y,n,n,n,n,n,n,?,y,y,y
republican,n,y,n,y,y,y,n,n,n,n,n,n,y,y,?,y
republican,n,y,n,y,y,y,n,n,n,n,n,y,y,y,n,y
democrat,y,y,y,n,n,n,y,y,y,n,n,n,n,n,?,?
republican,n,y,n,y,y,n,n,n,n,n,?,?,y,y,n,n
republican,n,y,n,y,y,y,n,n,n,n,y,?,y,y,?,?
democrat,n,y,y,n,n,n,y,y,y,n,n,n,y,n,?,?
democrat,y,y,y,n,n,y,y,y,?,y,y,?,n,n,y,?
republican,n,y,n,y,y,y,n,n,n,n,n,y,?,?,n,?
republican,n,y,n,y,y,y,n,n,n,y,n,y,y,?,n,?
democrat,y,n,y,n,n,y,n,y,?,y,y,y,?,n,n,y
democrat,y,?,y,n,n,n,y,y,y,n,n,n,y,n,y,y
republican,n,y,n,y,y,y,n,n,n,n,n,?,y,y,n,n
democrat,y,y,y,n,n,n,y,y,y,n,y,n,n,n,y,y
democrat,y,y,y,n,n,?,y,y,n,n,y,n,n,n,y,y
democrat,y,y,y,n,n,n,y,y,y,n,n,n,?,?,y,y
democrat,y,?,y,n,n,n,y,y,y,n,n,?,n,n,y,y
democrat,y,y,y,n,n,n,y,y,y,n,n,n,n,n,y,

In [0]:
# reading data into a dataframe

import pandas as pd
voting_data = pd.read_csv(voting_data_url, header=None)


In [3]:
voting_data.shape

(435, 17)

In [4]:
# cleaning data by adding named columns
voting_data.columns = ['party',	'handicapped-infants', 
                       'water-project',	'budget',	'physician-fee-freeze',	
                       'el-salvador-aid',	'religious-groups',	
                       'anti-satellite-ban', 'aid-to-contras', 'mx-missile',	
                       'immigration', 'synfuels', 'education',
                       'right-to-sue',	'crime', 'duty-free', 'south-africa']
voting_data.head()

,party,handicapped-infants,water-project,budget,physician-fee-freeze,el-salvador-aid,religious-groups,anti-satellite-ban,aid-to-contras,mx-missile,immigration,synfuels,education,right-to-sue,crime,duty-free,south-africa
0,republican,n,y,n,y,y,y,n,n,n,y,?,y,y,y,n,y
1,republican,n,y,n,y,y,y,n,n,n,n,n,y,y,y,n,?
2,democrat,?,y,y,?,y,y,n,n,n,n,y,n,y,y,n,n
3,democrat,n,y,y,n,?,y,n,n,n,n,y,n,y,n,n,y
4,democrat,y,y,y,n,y,y,n,n,n,n,y,?,y,y,y,y


In [5]:
voting_data.set_index('party')
voting_data.head()

,party,handicapped-infants,water-project,budget,physician-fee-freeze,el-salvador-aid,religious-groups,anti-satellite-ban,aid-to-contras,mx-missile,immigration,synfuels,education,right-to-sue,crime,duty-free,south-africa
0,republican,n,y,n,y,y,y,n,n,n,y,?,y,y,y,n,y
1,republican,n,y,n,y,y,y,n,n,n,n,n,y,y,y,n,?
2,democrat,?,y,y,?,y,y,n,n,n,n,y,n,y,y,n,n
3,democrat,n,y,y,n,?,y,n,n,n,n,y,n,y,n,n,y
4,democrat,y,y,y,n,y,y,n,n,n,n,y,?,y,y,y,y


In [6]:
# cleaning data by changing 'y' and 'n' to "1", and "0" for ease of use
voting_data_2 = voting_data.replace({'y': 1, 'n': 0})  
voting_data_2.head() 
                                           
                                      

,party,handicapped-infants,water-project,budget,physician-fee-freeze,el-salvador-aid,religious-groups,anti-satellite-ban,aid-to-contras,mx-missile,immigration,synfuels,education,right-to-sue,crime,duty-free,south-africa
0,republican,0,1,0,1,1,1,0,0,0,1,?,1,1,1,0,1
1,republican,0,1,0,1,1,1,0,0,0,0,0,1,1,1,0,?
2,democrat,?,1,1,?,1,1,0,0,0,0,1,0,1,1,0,0
3,democrat,0,1,1,0,?,1,0,0,0,0,1,0,1,0,0,1
4,democrat,1,1,1,0,1,1,0,0,0,0,1,?,1,1,1,1


In [7]:
#cleaning data by replacing "?" with NaN values

import numpy as np

voting_data_cleaned = voting_data_2.replace({'?': np.NaN})  
voting_data_cleaned.head() 

,party,handicapped-infants,water-project,budget,physician-fee-freeze,el-salvador-aid,religious-groups,anti-satellite-ban,aid-to-contras,mx-missile,immigration,synfuels,education,right-to-sue,crime,duty-free,south-africa
0,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,0.0,1.0
1,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,NaN
2,democrat,NaN,1.0,1.0,NaN,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0
3,democrat,0.0,1.0,1.0,0.0,NaN,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
4,democrat,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,1.0


In [8]:
#renaming voting_data_cleaned as df

df = voting_data_cleaned
df.head()

,party,handicapped-infants,water-project,budget,physician-fee-freeze,el-salvador-aid,religious-groups,anti-satellite-ban,aid-to-contras,mx-missile,immigration,synfuels,education,right-to-sue,crime,duty-free,south-africa
0,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,0.0,1.0
1,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,NaN
2,democrat,NaN,1.0,1.0,NaN,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0
3,democrat,0.0,1.0,1.0,0.0,NaN,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
4,democrat,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,1.0


In [9]:
# starting t-tests with separate dataframes by party affiliation
rep = df[df['party'] == 'republican']
rep.head()


,party,handicapped-infants,water-project,budget,physician-fee-freeze,el-salvador-aid,religious-groups,anti-satellite-ban,aid-to-contras,mx-missile,immigration,synfuels,education,right-to-sue,crime,duty-free,south-africa
0,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,0.0,1.0
1,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,NaN
7,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,NaN,1.0
8,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0
10,republican,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,1.0,1.0,0.0,0.0


In [10]:
dem = df[df['party'] == 'democrat']
dem.head()

,party,handicapped-infants,water-project,budget,physician-fee-freeze,el-salvador-aid,religious-groups,anti-satellite-ban,aid-to-contras,mx-missile,immigration,synfuels,education,right-to-sue,crime,duty-free,south-africa
2,democrat,NaN,1.0,1.0,NaN,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0
3,democrat,0.0,1.0,1.0,0.0,NaN,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
4,democrat,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,1.0
5,democrat,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0
6,democrat,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,1.0,1.0,1.0


In [30]:
# dropping the NaN values
dem['handicapped-infants'].value_counts(dropna=False)

1.0    156
0.0    102
NaN      9
Name: handicapped-infants, dtype: int64

**Pratice on One Sample t-Testing**

In [12]:
# calculating mean of voting
dem['handicapped-infants'].mean()

0.6046511627906976

In [0]:
# importing scipy and ttest 1 sample 
from scipy.stats import ttest_1samp, ttest_ind, ttest_ind_from_stats, ttest_rel

In [29]:
ttest_1samp(dem['handicapped-infants'], .5, nan_policy='omit')

Ttest_1sampResult(statistic=3.431373087696574, pvalue=0.000699612317167372)

In [15]:
ttest_1samp(dem['handicapped-infants'], 1, nan_policy='omit')

Ttest_1sampResult(statistic=-12.96296499796484, pvalue=6.590394568934029e-30)

In [16]:
ttest_1samp(dem['handicapped-infants'], 0, nan_policy='omit')

Ttest_1sampResult(statistic=19.825711173357988, pvalue=1.0391992873567661e-53)

In [17]:
ttest_1samp(dem['handicapped-infants'], .6, nan_policy='omit')

Ttest_1sampResult(statistic=0.15250547056429176, pvalue=0.8789079366662332)

Null hypothesis: The dem votes are neither particularly in favor or against this issue.

Alternative hypothesis: The democrats voted in favor of the issue

T statistic: 3.431

P-value:0.000699612317167372

Confidence interval - 99%, p-value < 0.01

Interpretation:

Due to calculating a t-statistic of T statistic: 3.431 which corresponds to a p-value of 0.000699612317167372, I failed to reject the null hypothesis that the democrats were neither particularly for or against this issue. 

In [32]:
# dropping the NaN values
dem['synfuels'].value_counts(dropna=False)

1.0    129
0.0    126
NaN     12
Name: synfuels, dtype: int64

In [33]:
# calculating one sampe t-test for 'synfuels'

ttest_1samp(dem['synfuels'], .5, nan_policy='omit')

Ttest_1sampResult(statistic=0.18751153526716705, pvalue=0.8514092819865365)

In [34]:
ttest_1samp(dem['synfuels'], 0, nan_policy='omit')

Ttest_1sampResult(statistic=16.125992032976423, pvalue=9.159793118867048e-41)

In [35]:
ttest_1samp(dem['synfuels'], .6, nan_policy='omit')

Ttest_1sampResult(statistic=-3.0001845642746834, pvalue=0.002966521634685216)

In [36]:
ttest_1samp(dem['synfuels'], 1, nan_policy='omit')

Ttest_1sampResult(statistic=-15.750968962442089, pvalue=1.839716417353916e-39)

Null hypothesis: The dem votes are neither particularly in favor or against this issue.

Alternative hypothesis: The democrats voted in favor of the issue

T statistic: 0.18751

P-value:0.85140

Confidence interval - 99%, p-value < 0.01

Interpretation:

Due to calculating a t-statistic of T statistic: 0.1875 which corresponds to a p-value of 0.8514092819865365, I  reject the null hypothesis that the democrats were neither particularly for or against this issue. 

**Two Sample t-Testing**

In [22]:
# calculating two sample tests for duty-free

print("Democratic Support: ", dem['duty-free'].mean())
print("Republican Support: ", rep['duty-free'].mean())

Democratic Support:  0.6374501992031872
Republican Support:  0.08974358974358974


In [23]:
ttest_ind(rep['duty-free'], dem['duty-free'], nan_policy='omit')

Ttest_indResult(statistic=-12.853146132542978, pvalue=5.997697174347365e-32)

1. Null hypothesis: The mean of dem support == The mean for republican support

2. Alternative hypothesis: The means are different(not the same level of support)

3. T statistic: 12.853

4. P-value: 0.000000000000000000000000000000005997

5. Confidence interval - 99%, p-value < 0.01

Interpretation: 

Due to calculating a t-statistic of 9.737 which corresponds to a p-value  of 0.000000000000000000000000000000005997, I would reject the null hypothesis, that there is the same level of support for this bill among republicans and democrats. (Ie. the democrats support this bill more than republicans)

In [24]:
#calculating two sample tests for el-salvador-aid
print("Democratic Support: ", dem['el-salvador-aid'].mean())
print("Republican Support: ", rep['el-salvador-aid'].mean())

Democratic Support:  0.21568627450980393
Republican Support:  0.9515151515151515


In [25]:
ttest_ind(rep['el-salvador-aid'], dem['el-salvador-aid'], nan_policy='omit')

Ttest_indResult(statistic=21.13669261173219, pvalue=5.600520111729011e-68)

1. Null hypothesis: The mean of dem support == The mean for republican support

2. Alternative hypothesis: The means are different(not the same level of support)

3. T statistic:21.136

4. P-value: 0.5600520111729011e-68

5. Confidence interval - 99%, p-value < 0.01

Interpretation: 

Due to calculating a t-statistic of 21.136 which corresponds to a p-value  of 0.5600520111729011e-68, I would reject the null hypothesis, that there is the same level of support for this bill among republicans and democrats. (ie republicans support this bill more than democrats)

In [26]:
#calculating two sample tests for mx-missile

print("Democratic Support: ", dem['anti-satellite-ban'].mean())
print("Republican Support: ", rep['anti-satellite-ban'].mean())

Democratic Support:  0.7722007722007722
Republican Support:  0.24074074074074073


In [27]:
ttest_ind(rep['south-africa'], dem['anti-satellite-ban'], nan_policy='omit')

Ttest_indResult(statistic=-2.511474145475195, pvalue=0.012413017063908115)

1. Null hypothesis: The mean of dem support == The mean for republican support

2. Alternative hypothesis: The means are different(not the same level of support)

3. T statistic: -2.511474145475195

4. P-value: 0.0124130170

5. Confidence interval - 99% p-value < 0.01

Interpretation: 

Due to calculating a t-statistic of  -2.511  which corresponds to a p-value  of 0.012, I failed to reject the null hypothesis, that there is the same level of support for this bill among republicans and democrats. (ie there is not a great difference in support between republicans and democrats supporting this issue)